In [ ]:
import geopandas as gpd 
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import dssg.dataio.osm_to_voronoi_mapping as ovm
data_dir = os.environ.get("DATA_DIR")
india_voronoi_gpd = gpd.read_file(data_dir + "voronoi/voronoi-cleaned.shp")

In [ ]:
india_voronoi_gpd.head()

In [ ]:
import dssg.dataio.osm_data_extraction as ode
india_shape = os.environ.get("DATA_DIR") + "/gadm36_shp/gadm36_IND_2.shp"

In [ ]:
import matplotlib.pyplot as plt 
%matplotlib inline
india_gpd = gpd.read_file(india_shape)
araria_gdf = ode.extract_district_dataframe(india_gpd, 'Araria')

In [ ]:
ovm.plot_district_voronoi(india_voronoi_gpd, "India")

In [ ]:
araria_voronoi_gpd = ovm.extract_district_voronoi(india_voronoi_gpd, araria_gdf)

In [ ]:
ovm.plot_district_voronoi(araria_voronoi_gpd, "Araria")

In [ ]:
print("The clipped data have fewer line objects (represented by rows):", araria_voronoi_gpd.shape, india_voronoi_gpd.shape)

In [ ]:
araria_voronoi_gpd.head()